In [5]:
require 'daru'
require 'nyaplot'
require 'numo/narray'

N1 = 20
Mu1 = [15, 10]

N2 = 30
Mu2 = [0, 0]

Variances = [15, 30]

class Array
  def mean
    self.inject(:+) / self.size.to_f
  end
end

def normal_rand(mu = 0, sigma = 1.0)
  random = Random.new
  (Math.sqrt(-2 * Math.log(random.rand)) * Math.sin(2 * Math::PI * random.rand) * sigma) + mu
end

# データセット {x_n,y_n,type_n} を用意
def prepare_dataset(variance)
  sigma = Math.sqrt(variance)

  df1 = N1.times.map do
    [normal_rand(Mu1[0], sigma), normal_rand(Mu1[1], sigma)]
  end
  df1 = df1.transpose
  df1 = Daru::DataFrame.new(x: df1[0], y: df1[1], type: Array.new(N1).fill(1))

  df2 = N2.times.map do
    [normal_rand(Mu2[0], sigma), normal_rand(Mu2[1], sigma)]
  end
  df2 = df2.transpose
  df2 = Daru::DataFrame.new(x: df2[0], y: df2[1], type: Array.new(N2).fill(-1))

  df = df1.concat(df2)
  df = df.reindex(Daru::Index.new(df.index.to_a.shuffle))
  df[:index] = (N1 + N2).times.to_a
  df.set_index(:index)
end

# Perceptronのアルゴリズム（確率的勾配降下法）を実行
def run_simulation(variance)
  data_graph = Nyaplot::Plot.new
  param_graph = Nyaplot::Plot.new

  train_set = prepare_dataset(variance)
  train_set1 = train_set.filter_rows {|row| row[:type] == 1 }
  train_set2 = train_set.filter_rows {|row| row[:type] == -1 }
  ymin = train_set.y.min - 5
  xmin = train_set.x.min - 5
  ymax = train_set.y.max + 10
  xmax = train_set.x.max + 10
  data_graph.configure do
    x_label('')
    y_label('')
    xrange([xmin, xmax])
    yrange([ymin, ymax])
    legend(true)
    height(300)
    width(490)
  end

  scatter_true = data_graph.add_with_df(train_set1.to_nyaplotdf, :scatter, :x, :y)
  scatter_true.color('green')
  scatter_true.title('1')
  scatter_false = data_graph.add_with_df(train_set2.to_nyaplotdf, :scatter, :x, :y)
  scatter_false.color('orange')
  scatter_false.title('-1')
  
  # パラメータの初期値とbias項の設定
  w0 = 0.0
  w1 = 0.0
  w2 = 0.0
  bias = 0.5 * (train_set.x.mean + train_set.y.mean)
    
  # Iterationを30回実施
  paramhist = Daru::DataFrame.new(w0: [w0], w1: [w1], w2: [w2])
  30.times do
    train_set.each_row do |point|
      x = point.x
      y = point.y
      type = point[:type]
      if type * (w0*bias + w1*x + w2*y) <= 0
        w0 += type * bias
        w1 += type * x
        w2 += type * y
      end
    end
    paramhist.add_row(Daru::Vector.new([w0, w1, w2], index: [:w0, :w1, :w2]))
  end
  
  # 判定誤差の計算
  err = 0
  train_set.each_row do |point|
    x = point.x
    y = point.y
    type = point[:type]
    if type * (w0*bias + w1*x + w2*y) <= 0
      err += 1
    end
  end
  
  err_rate = err * 100 / train_set.size
  
  # 結果の表示
  linex = Numo::NArray[*(xmin.to_i-5..xmax.to_i+4).to_a]
  liney = -linex * w1 / w2 - bias * w0 / w2
  label = "ERR %.2f%%" % err_rate
  line_err = data_graph.add(:line, linex.cast_to(Numo::Int64).to_a, liney.cast_to(Numo::Int64).to_a)
  line_err.title(label)
  line_err.color('red')
  line_w0 = param_graph.add(:line, paramhist.index.to_a, paramhist.w0)
  line_w0.title('w0')
  line_w0.color('blue')
  line_w1 = param_graph.add(:line, paramhist.index.to_a, paramhist.w1)
  line_w1.title('w1')
  line_w1.color('green')
  line_w2 = param_graph.add(:line, paramhist.index.to_a, paramhist.w2)
  line_w2.title('w2')
  line_w2.color('red')
  
  param_graph.configure do
    x_label('')
    y_label('')
    legend(true)
    height(300)
    width(490)
  end

  [data_graph, param_graph]
end

fig = Nyaplot::Frame.new

Variances.each do |variance|
  data_graph, param_graph = run_simulation(variance)
  fig.add(data_graph)
  fig.add(param_graph)
end

fig.show

<main>:4: warning: already initialized constant N1
<main>:4: warning: previous definition of N1 was here
<main>:5: warning: already initialized constant Mu1
<main>:5: warning: previous definition of Mu1 was here
<main>:7: warning: already initialized constant N2
<main>:7: warning: previous definition of N2 was here
<main>:8: warning: already initialized constant Mu2
<main>:8: warning: previous definition of Mu2 was here
<main>:10: warning: already initialized constant Variances
<main>:10: warning: previous definition of Variances was here


#<Nyaplot::Frame:0x007f7235939f28 @properties={:panes=>[#<Nyaplot::Plot:0x007f7235939de8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f7235c33d00 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"green", :title=>"1"}, :data=>"8b558860-a49a-42f1-ad42-b2c1aa8a0453"}, @xrange=[8.797218655320995, 19.162957024682385], @yrange=[4.5247504514056045, 15.478472658380173]>, #<Nyaplot::Diagram:0x007f7235c09230 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"orange", :title=>"-1"}, :data=>"85519ea4-70e8-4002-b240-f1eee877f928"}, @xrange=[-6.35991269603446, 3.9078149123744113], @yrange=[-4.685324258232235, 6.881378488461043]>, #<Nyaplot::Diagram:0x007f7235a83c80 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 0.00%", :color=>"red"}, :data=>"aa0e03ef-7da6-4e89-a11c-b4b847b82ea7"}, @xrange=[-16, 33], @yrange=[-63, 51]>], :options=>{:x_label=>"", :y_label=>"", :xrange=>[-11.35991269603446, 29.162957024682385], :yrange=>[-9.685324258232235, 25.478472658380173], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f7235939c80 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f7235a7ce80 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"w0", :color=>"blue"}, :data=>"81ae2882-67e5-4379-b176-599279236845"}, @xrange=[0, 30], @yrange=[-4.839888301280314, 0.0]>, #<Nyaplot::Diagram:0x007f7235a40f98 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"w1", :color=>"green"}, :data=>"c2d908a1-6d5b-49cc-a3e2-00bc3dd7881a"}, @xrange=[0, 30], @yrange=[0.0, 4.00293305399947]>, #<Nyaplot::Diagram:0x007f7235a2d588 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"w2", :color=>"red"}, :data=>"f5b1b556-5da8-412a-b66e-343715e6b571"}, @xrange=[0, 30], @yrange=[0.0, 1.7129939361739042]>], :options=>{:x_label=>"", :y_label=>"", :legend=>true, :height=>300, :width=>490, :zoom=>true, :xrange=>[0, 30], :yrange=>[-4.839888301280314, 4.00293305399947]}}>, #<Nyaplot::Plot:0x007f7235a2c958 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f72351393f0 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"green", :title=>"1"}, :data=>"468cb0bd-6fd7-4f93-a977-a14294cd3492"}, @xrange=[5.0289347019801, 34.5928158811065], @yrange=[-1.1207367386370493, 19.927973126378447]>, #<Nyaplot::Diagram:0x007f7234f15df8 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"orange", :title=>"-1"}, :data=>"006cfc46-c08a-4b60-80e0-a6be51ef166e"}, @xrange=[-9.886310433272591, 8.963759379804962], @yrange=[-13.027593839817, 13.863180383675985]>, #<Nyaplot::Diagram:0x007f7235b2dd48 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 6.00%", :color=>"red"}, :data=>"0317d3f8-7807-4a58-897a-2bf491ef3613"}, @xrange=[-19, 48], @yrange=[-203, 132]>], :options=>{:x_label=>"", :y_label=>"", :xrange=>[-14.886310433272591, 44.5928158811065], :yrange=>[-18.027593839817, 29.927973126378447], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f7235a2c728 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f7235b24298 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"w0", :color=>"blue"}, :data=>"f7d01f7c-10f0-4320-8198-7ece298470ce"}, @xrange=[0, 30], @yrange=[-60.40141449972084, 0.0]>, #<Nyaplot::Diagram:0x007f7235b0a280 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"w1", :color=>"green"}, :data=>"77404b00-25bd-45f5-947d-f874358e12d3"}, @xrange=[0, 30], @yrange=[0.0, 44.909670550213804]>, #<Nyaplot::Diagram:0x007f7235afea70 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"w2", :color=>"red"}, :data=>"c53a870e-6eb1-4ece-b14e-3d48c8c01b1f"}, @xrange=[0, 30], @yrange=[0.0, 20.499281340748784]>], :options=>{:x_label=>"", :y_label=>"", :legend=>true, :height=>300, :width=>490, :zoom=>true, :xrange=>[0, 30], :yrange=>[-60.40141449972084, 44.909670550213804]}}>], :data=>{"8b558860-a49a-42f1-ad42-b2c1aa8a0453"=>#<Nyaplo